[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/boostcourse-ds-510/blob/master/k-beauty-oversea-online-sale-output.ipynb)

## 국가(대륙)별/상품군별 온라인쇼핑 해외직접판매액
* 국가통계포털 : http://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1KE10081&vw_cd=MT_ZTITLE&list_id=JF&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE

### K-Beauty는 성장하고 있을까? 해외 직접판매를 한다면 어느 국가로 판매전략을 세우면 좋을까?
* K-Beauty란? [K-Beauty - Wikipedia](https://en.wikipedia.org/wiki/K-Beauty)
* e : 추정치, p : 잠정치, - : 자료없음, ... : 미상자료, x : 비밀보호, ▽ : 시계열 불연 ( 단위 : 백만원 )

## 필요 라이브러리 가져오기

## 한글폰트 설정
* https://github.com/ychoi-kr/koreanize-matplotlib

In [ ]:
# 한글폰트 사용을 위해 설치
# 아래 모듈을 설치하고 불러오면 별도의 한글폰트 설정이 필요 없습니다.
# colab 에서는 아래 모듈을 설치하고 불러와서 사용하세요. 

# !pip install koreanize-matplotlib

# import koreanize_matplotlib

In [ ]:
# os 별로 폰트를 다르게 설정해 줍니다.
    
# 레티나 설정을 해주면 글씨가 좀 더 선명하게 보입니다.
# 폰트의 주변이 흐릿하게 보이는 것을 방지합니다.
%config InlineBackend.figure_format = 'retina'

## 데이터 로드하기

In [ ]:
# %ls data

In [ ]:
from glob import glob

# glob("data/*.csv")

In [ ]:
# df_raw = pd.read_csv("data/국가_대륙_별_상품군별_온라인쇼핑_해외직접판매액_202002.csv", 
#                      encoding="cp949")
df_raw = pd.read_csv("https://raw.githubusercontent.com/corazzon/boostcourse-ds-510/master/data/online_oversea_sale.csv.zip", 
                     encoding="cp949")
df_raw.shape

In [ ]:
# "국가(대륙)별" 데이터 빈도수 세기


In [ ]:
# 미국 데이터만 따로 보기


## 분석과 시각화를 위한 tidy data 만들기
* https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

## 데이터 전처리
### 기간에서 연도를 분리하기

In [ ]:
# df["연도"]

In [ ]:
# df["분기"]

### 금액을 수치데이터로 표현하기 위해 데이터 타입 변경하기

In [ ]:
# df["million"]

### 필요없는 데이터 제거하기

In [ ]:
# 합계 데이터는 따로 구할 수 있기 때문에 전체 데이터에서 제거합니다.
# df["국가(대륙)별"] != "합계"
# df["상품군별"] != "합계"
# df["판매유형별"] != "계"


## K-Beauty 시각화
### 전체 상품군 판매액

In [ ]:
plt.figure(figsize=(15, 4))


In [ ]:
# df_sub

In [ ]:
# sns.relplot


### 화장품의 온라인쇼핑 해외직접판매액

In [ ]:
# df_cosmetic

### 의류 및 패션관련 상품 온라인쇼핑 해외직접판매액

In [ ]:
# df_fashion 

### 데이터 집계하기

### 연산결과를 시각적으로 보기

## 전체 상품군별로 온라인쇼핑 해외직접판매액은 증가했을까?

In [ ]:
plt.figure(figsize=(15, 4))


* lengend를 그래프의 밖에 그리기 : [matplotlib - Move legend outside figure in seaborn tsplot - Stack Overflow](https://stackoverflow.com/questions/30490740/move-legend-outside-figure-in-seaborn-tsplot)

In [ ]:
plt.figure(figsize=(15, 4))
